## College Recommendation System
This is an attempt to use the information at College Scorecard, the usgov data tracking system for college stats, to recommend potential target colleges for students based on their current school choices.

In [196]:
import pandas as pd
import json
import pickle

df = pd.read_csv('big_college.csv')

/Users/aaronlee/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (9,1547,1589,1595,1596,1597,1598,1601,1602,1608,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1640,1643,1648,1653,1657,1664,1665,1666,1669,1670,1674,1679,1683,1688,1689,1690,1691,1692,1697,1700,1702,1703,1704,1705,1706,1707,1725,1726,1727,1728,1729,1743,1815,1816,1817,1818,1823,1824,1830,1831,1846,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1909,1910,1911,1912,1913,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1983,1984,2376,2377) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [206]:
df = df[df['CCSIZSET']>1]  # alternativly, use df[df['PREDDEG']>1]  (degree seeking majority)
len(df)

3673

In [207]:
#pd.options.display.max_rows = 4000
pd.options.display.max_seq_items = 4000
df.columns


Index(['UNITID', 'OPEID', 'OPEID6', 'INSTNM', 'CITY', 'STABBR', 'ZIP',
       'ACCREDAGENCY', 'INSTURL', 'NPCURL', 'SCH_DEG', 'HCM2', 'MAIN',
       'NUMBRANCH', 'PREDDEG', 'HIGHDEG', 'CONTROL', 'ST_FIPS', 'REGION',
       'LOCALE', 'LOCALE2', 'LATITUDE', 'LONGITUDE', 'CCBASIC', 'CCUGPROF',
       'CCSIZSET', 'HBCU', 'PBI', 'ANNHI', 'TRIBAL', 'AANAPII', 'HSI', 'NANTI',
       'MENONLY', 'WOMENONLY', 'RELAFFIL', 'ADM_RATE', 'ADM_RATE_ALL',
       'SATVR25', 'SATVR75', 'SATMT25', 'SATMT75', 'SATWR25', 'SATWR75',
       'SATVRMID', 'SATMTMID', 'SATWRMID', 'ACTCM25', 'ACTCM75', 'ACTEN25',
       'ACTEN75', 'ACTMT25', 'ACTMT75', 'ACTWR25', 'ACTWR75', 'ACTCMMID',
       'ACTENMID', 'ACTMTMID', 'ACTWRMID', 'SAT_AVG', 'SAT_AVG_ALL', 'PCIP01',
       'PCIP03', 'PCIP04', 'PCIP05', 'PCIP09', 'PCIP10', 'PCIP11', 'PCIP12',
       'PCIP13', 'PCIP14', 'PCIP15', 'PCIP16', 'PCIP19', 'PCIP22', 'PCIP23',
       'PCIP24', 'PCIP25', 'PCIP26', 'PCIP27', 'PCIP29', 'PCIP30', 'PCIP31',
       'PCIP38', 'PCIP39

In [208]:
df = df[df['DISTANCEONLY']==0] # lose the online schools
df = df[df['PREDDEG'] == 3] # lose the 2 year schools and community colleges (lose 1600)

df.reset_index(inplace=True)
len(df)


2001

In [209]:
ident = [            
    'UNITID', # index
    'INSTNM', # for identification
    'CITY',
    'STABBR',
    'INSTURL',
    'UGDS',
    'CONTROL',
    'COSTT4_A',
    'ADM_RATE',
        ]

cat = [#'STABBR', #state
       'REGION', #midwest, northesst etc.
       'CONTROL', # public=0, private=1, private nonprofit=2  CAT
       'HBCU',
       'PBI',
       'ANNHI',
       'TRIBAL',
       'AANAPII',
       'HSI',
       'NANTI',
       'MENONLY',
       'WOMENONLY',
       #'RELAFFIL',
      ]

cardinal = [
    'LOCALE', # how rural CARD
    'CCBASIC', # focus of school CARD
    'CCUGPROF', # focus of school CARD   
    'CCSIZSET',  # size of school
]

continuous = [
    'ADM_RATE',
    'ACTCMMID',
    'ACTENMID',
    'ACTMTMID',
    'ACTWRMID',
    'SATMTMID',
    'SATWRMID',
    'SAT_AVG',
#     'PCIP01',
#     'PCIP03', # percentage of each degree type (are we an engineering school, english, journalism etc.)
#     'PCIP04',
#     'PCIP05',
    'PCIP09', # JOURNALISM
#     'PCIP10',
#    'PCIP11', # COMPSCI
#     'PCIP12',
    'PCIP13', # EDUCATION
    'PCIP14', # ENGINEERING
#     'PCIP15',
#     'PCIP16',
#     'PCIP19',
#     'PCIP22',
#     'PCIP23',
    'PCIP24', # LIBARTS
#     'PCIP25',
#     'PCIP26',
#     'PCIP27',
#     'PCIP29',
#     'PCIP30',
#     'PCIP31',
#     'PCIP38',
#     'PCIP39',
#     'PCIP40',
#     'PCIP41',
#     'PCIP42',
#     'PCIP43',
#     'PCIP44',
#     'PCIP45',
#     'PCIP46',
#     'PCIP47',
#     'PCIP48',
#     'PCIP49',
    'PCIP50', # PERFORMING ARTS
#     'PCIP51',
#     'PCIP52',
#     'PCIP54',
    'UGDS', # student population
    'COSTT4_A', # cost attendance
    #'PCTPELL', # pell grant percent
    #'C200_4', # completeion rate for 4yr students
    #'FAMINC', #family income
]

df_final = df[cardinal+continuous]
df_dummies = pd.get_dummies(df[cat])

df_final = pd.concat([df_final, df_dummies], axis=1)
df_final_named = df[ident]


In [210]:
pd.options.display.max_columns = 300
df_final.shape
df_final.head(50)

,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,ADM_RATE,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SATMTMID,SATWRMID,SAT_AVG,PCIP09,PCIP13,PCIP14,PCIP24,PCIP50,UGDS,COSTT4_A,REGION,CONTROL,HBCU,PBI,ANNHI,TRIBAL,AANAPII,HSI,NANTI,MENONLY,WOMENONLY
0,12.0,18.0,10.0,14.0,0.8986,18.0,17.0,17.0,NaN,465.0,414.0,957.0,0.0000,0.0710,0.1183,0.0473,0.0237,4990.0,22489.0,5,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12.0,15.0,9.0,15.0,0.9211,25.0,27.0,23.0,NaN,555.0,NaN,1220.0,0.0375,0.0717,0.0813,0.0351,0.0339,13186.0,24347.0,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12.0,16.0,15.0,12.0,0.8087,28.0,30.0,27.0,NaN,565.0,NaN,1314.0,0.0194,0.0283,0.2892,0.0000,0.0380,7458.0,23441.0,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12.0,19.0,10.0,14.0,0.9774,18.0,17.0,17.0,NaN,465.0,NaN,972.0,0.0892,0.1169,0.0000,0.0000,0.0585,3903.0,21476.0,5,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12.0,15.0,14.0,16.0,0.5906,27.0,29.0,25.0,8.0,580.0,540.0,1252.0,0.0973,0.0643,0.1160,0.0000,0.0244,32177.0,29424.0,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,31.0,22.0,5.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.3333,0.0000,0.0461,0.0303,2732.0,NaN,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,12.0,18.0,9.0,13.0,0.9281,21.0,21.0,19.0,NaN,525.0,NaN,1084.0,0.0339,0.0854,0.0000,0.0027,0.0298,4529.0,18291.0,5,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,13.0,15.0,14.0,15.0,0.7543,28.0,29.0,26.0,8.0,615.0,570.0,1289.0,0.0566,0.0602,0.1911,0.0000,0.0296,24147.0,31282.0,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,12.0,21.0,14.0,11.0,0.5666,26.0,27.0,23.0,NaN,560.0,NaN,1232.0,0.0284,0.0355,0.0248,0.0000,0.1099,1265.0,52176.0,5,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,12.0,20.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.0000,0.0000,321.0,29309.0,5,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [211]:
#df_final.fillna('')


In [212]:
pd.options.display.max_rows = 200
df_final.isna().sum()
df_final.fillna(0, inplace=True)
# df_final['FAMINC'] = df_final['FAMINC'].isna
# df_final['FAMINC'] = df_final['FAMINC'].astype(float)


In [213]:
df_final.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2001 entries, 0 to 2000
Data columns (total 30 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   LOCALE     2001 non-null   float64
 1   CCBASIC    2001 non-null   float64
 2   CCUGPROF   2001 non-null   float64
 3   CCSIZSET   2001 non-null   float64
 4   ADM_RATE   2001 non-null   float64
 5   ACTCMMID   2001 non-null   float64
 6   ACTENMID   2001 non-null   float64
 7   ACTMTMID   2001 non-null   float64
 8   ACTWRMID   2001 non-null   float64
 9   SATMTMID   2001 non-null   float64
 10  SATWRMID   2001 non-null   float64
 11  SAT_AVG    2001 non-null   float64
 12  PCIP09     2001 non-null   float64
 13  PCIP13     2001 non-null   float64
 14  PCIP14     2001 non-null   float64
 15  PCIP24     2001 non-null   float64
 16  PCIP50     2001 non-null   float64
 17  UGDS       2001 non-null   float64
 18  COSTT4_A   2001 non-null   float64
 19  REGION     2001 non-null   int64  
 20  CONTROL 

In [214]:
df_final['ACTWRMID'] = df_final['ACTWRMID'].fillna(6)
df_final['ACTWRMID'] = df_final['ACTWRMID'].fillna(6)

# df_final['RELAFFIL'].unique()
# df_final['RELAFFIL'] = df_final['RELAFFIL'].fillna(0)

#df_final.max()
df_final.isna().sum().max()


0

## Normalize my data


In [215]:
import scipy
import pandas as pd
import numpy as np
from sklearn import preprocessing


scaler = preprocessing.MinMaxScaler()

scaled_data = scaler.fit_transform(df_final)
df_scaled = pd.DataFrame(scaled_data, columns= df_final.columns)

# # WEIGHTING
# weights = [
#             ('ADM_RATE', 50),
#             ('REGION', 1000),
#             ('UGDS', 50),
#             ('CONTROL', 5),
#             ('COSTT4_A', 5),
#             ('MENONLY', 50),
#             ('WOMENONLY', 50),
#             ('HBCU', 100),
#             ('CCBASIC', 10),
#             ('CCUGPROF', 10),
#             ('CCSIZSET', 20),
#             ('ACTCMMID', 50),
#             ('SATMTMID', 50),
#             ('SATWRMID', 50),
#             ('ACTENMID', 50),
#             ('ACTMTMID', 50),
#             ('ACTWRMID', 50)
#             ]

# for col, weight in weights:
#     df_scaled[col] = df_scaled[col] * weight
    
df_scaled.head()



,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,ADM_RATE,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SATMTMID,SATWRMID,SAT_AVG,PCIP09,PCIP13,PCIP14,PCIP24,PCIP50,UGDS,COSTT4_A,REGION,CONTROL,HBCU,PBI,ANNHI,TRIBAL,AANAPII,HSI,NANTI,MENONLY,WOMENONLY
0,0.03125,0.166667,0.666667,0.666667,0.8986,0.500000,0.472222,0.472222,0.000000,0.584906,0.541176,0.611111,0.000000,0.0710,0.1183,0.0473,0.0237,0.062243,0.296943,0.555556,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.03125,0.000000,0.600000,0.750000,0.9211,0.694444,0.750000,0.638889,0.000000,0.698113,0.000000,0.779055,0.105693,0.0717,0.0813,0.0351,0.0339,0.164475,0.321476,0.555556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.03125,0.055556,1.000000,0.500000,0.8087,0.777778,0.833333,0.750000,0.000000,0.710692,0.000000,0.839080,0.054679,0.0283,0.2892,0.0000,0.0380,0.093027,0.309513,0.555556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.03125,0.222222,0.666667,0.666667,0.9774,0.500000,0.472222,0.472222,0.000000,0.584906,0.000000,0.620690,0.251409,0.1169,0.0000,0.0000,0.0585,0.048684,0.283568,0.555556,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.03125,0.000000,0.933333,0.833333,0.5906,0.750000,0.805556,0.694444,0.666667,0.729560,0.705882,0.799489,0.274239,0.0643,0.1160,0.0000,0.0244,0.401360,0.388513,0.555556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [216]:
import pickle
pickle.dump( df_scaled, open( "scaled_df.pkl", "wb" ))
pickle.dump(df_final_named, open('df_final_names.pkl', 'wb'))
df_final_named

,UNITID,INSTNM,CITY,STABBR,INSTURL,UGDS,CONTROL,COSTT4_A,ADM_RATE
0,100654,Alabama A & M University,Normal,AL,www.aamu.edu/,4990.0,1,22489.0,0.8986
1,100663,University of Alabama at Birmingham,Birmingham,AL,https://www.uab.edu,13186.0,1,24347.0,0.9211
2,100706,University of Alabama in Huntsville,Huntsville,AL,www.uah.edu,7458.0,1,23441.0,0.8087
3,100724,Alabama State University,Montgomery,AL,www.alasu.edu,3903.0,1,21476.0,0.9774
4,100751,The University of Alabama,Tuscaloosa,AL,www.ua.edu/,32177.0,1,29424.0,0.5906
...,...,...,...,...,...,...,...,...,...
1996,490513,Bais Medrash Mayan Hatorah,Lakewood,NJ,https://baismedrashmayanhatorah.com,69.0,2,16050.0,0.6471
1997,490771,Purdue University Global-St. Louis,St. Louis,MO,https://www.purdueglobal.edu/campus-locations/...,232.0,1,NaN,NaN
1998,490805,Purdue University Northwest,Hammond,IN,www.pnw.edu,8244.0,1,16949.0,0.3108
1999,490850,East-West University,Keshena,WI,www.eastwest.edu,11.0,2,40100.0,NaN


# Run a test


In [217]:
college_name = 'University of Florida'
college_id = df[df['INSTNM']==college_name] # test it out
college_id = college_id.index.to_list()[0]


test_college = df_scaled.iloc[[college_id]]
#test_college_scaled = scaler.transform(test_college)
#print(test_college_scaled)



ary = scipy.spatial.distance.cdist(df_scaled, test_college, metric='euclidean')

results = df_final.copy()
results['dist'] = ary


results = pd.concat([df_final_named, results], axis=1)
results

,UNITID,INSTNM,CITY,STABBR,INSTURL,UGDS,CONTROL,COSTT4_A,ADM_RATE,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,ADM_RATE,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SATMTMID,SATWRMID,SAT_AVG,PCIP09,PCIP13,PCIP14,PCIP24,PCIP50,UGDS,COSTT4_A,REGION,CONTROL,HBCU,PBI,ANNHI,TRIBAL,AANAPII,HSI,NANTI,MENONLY,WOMENONLY,dist
0,100654,Alabama A & M University,Normal,AL,www.aamu.edu/,4990.0,1,22489.0,0.8986,12.0,18.0,10.0,14.0,0.8986,18.0,17.0,17.0,0.0,465.0,414.0,957.0,0.0000,0.0710,0.1183,0.0473,0.0237,4990.0,22489.0,5,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.652561
1,100663,University of Alabama at Birmingham,Birmingham,AL,https://www.uab.edu,13186.0,1,24347.0,0.9211,12.0,15.0,9.0,15.0,0.9211,25.0,27.0,23.0,0.0,555.0,0.0,1220.0,0.0375,0.0717,0.0813,0.0351,0.0339,13186.0,24347.0,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.357793
2,100706,University of Alabama in Huntsville,Huntsville,AL,www.uah.edu,7458.0,1,23441.0,0.8087,12.0,16.0,15.0,12.0,0.8087,28.0,30.0,27.0,0.0,565.0,0.0,1314.0,0.0194,0.0283,0.2892,0.0000,0.0380,7458.0,23441.0,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.307624
3,100724,Alabama State University,Montgomery,AL,www.alasu.edu,3903.0,1,21476.0,0.9774,12.0,19.0,10.0,14.0,0.9774,18.0,17.0,17.0,0.0,465.0,0.0,972.0,0.0892,0.1169,0.0000,0.0000,0.0585,3903.0,21476.0,5,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.845078
4,100751,The University of Alabama,Tuscaloosa,AL,www.ua.edu/,32177.0,1,29424.0,0.5906,12.0,15.0,14.0,16.0,0.5906,27.0,29.0,25.0,8.0,580.0,540.0,1252.0,0.0973,0.0643,0.1160,0.0000,0.0244,32177.0,29424.0,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.345954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,490513,Bais Medrash Mayan Hatorah,Lakewood,NJ,https://baismedrashmayanhatorah.com,69.0,2,16050.0,0.6471,13.0,24.0,11.0,8.0,0.6471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,69.0,16050.0,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.483009
1997,490771,Purdue University Global-St. Louis,St. Louis,MO,https://www.purdueglobal.edu/campus-locations/...,232.0,1,NaN,NaN,21.0,22.0,5.0,6.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,232.0,0.0,4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.563803
1998,490805,Purdue University Northwest,Hammond,IN,www.pnw.edu,8244.0,1,16949.0,0.3108,21.0,18.0,9.0,12.0,0.3108,22.0,21.0,22.0,0.0,490.0,0.0,1022.0,0.0357,0.0504,0.0758,0.0171,0.0000,8244.0,16949.0,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.424874
1999,490850,East-West University,Keshena,WI,www.eastwest.edu,11.0,2,40100.0,NaN,42.0,21.0,10.0,6.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,11.0,40100.0,3,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.876644


dist_df is the wrong size?????

In [218]:
print(len(new_df), len(df_final_named), len(dist_df))

NameError: name 'new_df' is not defined

In [219]:
closest = results.sort_values(by='dist')
closest.head(50)

,UNITID,INSTNM,CITY,STABBR,INSTURL,UGDS,CONTROL,COSTT4_A,ADM_RATE,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,ADM_RATE,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SATMTMID,SATWRMID,SAT_AVG,PCIP09,PCIP13,PCIP14,PCIP24,PCIP50,UGDS,COSTT4_A,REGION,CONTROL,HBCU,PBI,ANNHI,TRIBAL,AANAPII,HSI,NANTI,MENONLY,WOMENONLY,dist
242,134130,University of Florida,Gainesville,FL,www.ufl.edu/,34564.0,1,21034.0,0.3875,12.0,15.0,15.0,16.0,0.3875,30.0,30.0,28.0,9.0,685.0,620.0,1362.0,0.0795,0.0184,0.1280,0.0337,0.0274,34564.0,21034.0,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1514,228778,The University of Texas at Austin,Austin,TX,www.utexas.edu,40329.0,1,25228.0,0.3852,11.0,15.0,15.0,15.0,0.3852,30.0,31.0,29.0,9.0,685.0,625.0,1367.0,0.1010,0.0000,0.1099,0.0119,0.0227,40329.0,25228.0,6,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.186926
290,139959,University of Georgia,Athens,GA,www.uga.edu,29474.0,1,26252.0,0.4863,12.0,15.0,15.0,16.0,0.4863,30.0,31.0,28.0,7.0,660.0,610.0,1345.0,0.1070,0.0391,0.0355,0.0023,0.0355,29474.0,26252.0,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.253792
1084,199193,North Carolina State University at Raleigh,Raleigh,NC,www.ncsu.edu,23708.0,1,23600.0,0.4685,11.0,15.0,15.0,16.0,0.4685,29.0,30.0,28.0,8.0,670.0,585.0,1336.0,0.0391,0.0236,0.2633,0.0052,0.0194,23708.0,23600.0,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.263890
1668,240444,University of Wisconsin-Madison,Madison,WI,www.wisc.edu,30360.0,1,25497.0,0.5173,11.0,15.0,14.0,16.0,0.5173,30.0,31.0,29.0,9.0,700.0,640.0,1365.0,0.0590,0.0280,0.1065,0.0010,0.0229,30360.0,25497.0,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.292306
1161,204796,Ohio State University-Main Campus,Columbus,OH,www.osu.edu/,45769.0,1,26555.0,0.5197,11.0,15.0,15.0,16.0,0.5197,30.0,30.0,30.0,9.0,705.0,610.0,1359.0,0.0504,0.0370,0.1451,0.0011,0.0225,45769.0,26555.0,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.324084
1079,199120,University of North Carolina at Chapel Hill,Chapel Hill,NC,www.unc.edu,18946.0,1,25527.0,0.2271,13.0,15.0,14.0,17.0,0.2271,31.0,32.0,30.0,9.0,700.0,645.0,1398.0,0.1085,0.0073,0.0129,0.0052,0.0224,18946.0,25527.0,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.330682
4,100751,The University of Alabama,Tuscaloosa,AL,www.ua.edu/,32177.0,1,29424.0,0.5906,12.0,15.0,14.0,16.0,0.5906,27.0,29.0,25.0,8.0,580.0,540.0,1252.0,0.0973,0.0643,0.1160,0.0000,0.0244,32177.0,29424.0,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.345954
1615,236948,University of Washington-Seattle Campus,Seattle,WA,www.washington.edu,30905.0,1,25275.0,0.4869,11.0,15.0,14.0,16.0,0.4869,30.0,30.0,30.0,9.0,690.0,590.0,1347.0,0.0600,0.0271,0.1018,0.0011,0.0330,30905.0,25275.0,8,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.375851
694,170976,University of Michigan-Ann Arbor,Ann Arbor,MI,https://umich.edu/,30079.0,1,29407.0,0.2283,12.0,15.0,14.0,16.0,0.2283,32.0,33.0,31.0,9.0,725.0,685.0,1434.0,0.0332,0.0087,0.1591,0.0081,0.0519,30079.0,29407.0,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.377771


In [220]:
colleges = list(closest['INSTNM'][1:20])
colleges

['The University of Texas at Austin',
 'University of Georgia',
 'North Carolina State University at Raleigh',
 'University of Wisconsin-Madison',
 'Ohio State University-Main Campus',
 'University of North Carolina at Chapel Hill',
 'The University of Alabama',
 'University of Washington-Seattle Campus',
 'University of Michigan-Ann Arbor',
 'University of Illinois at Urbana-Champaign',
 'The University of Texas at Dallas',
 'Pennsylvania State University-Main Campus',
 'University of South Florida-Main Campus',
 'Auburn University',
 'University of Cincinnati-Main Campus',
 'Texas A & M University-College Station',
 'University of Pittsburgh-Pittsburgh Campus',
 'Indiana University-Bloomington',
 'University of North Carolina Wilmington']

In [221]:
with open('colleges.txt', 'w') as outfile:
    json.dump(my_json, outfile)

In [222]:
df_final_named

,UNITID,INSTNM,CITY,STABBR,INSTURL,UGDS,CONTROL,COSTT4_A,ADM_RATE
0,100654,Alabama A & M University,Normal,AL,www.aamu.edu/,4990.0,1,22489.0,0.8986
1,100663,University of Alabama at Birmingham,Birmingham,AL,https://www.uab.edu,13186.0,1,24347.0,0.9211
2,100706,University of Alabama in Huntsville,Huntsville,AL,www.uah.edu,7458.0,1,23441.0,0.8087
3,100724,Alabama State University,Montgomery,AL,www.alasu.edu,3903.0,1,21476.0,0.9774
4,100751,The University of Alabama,Tuscaloosa,AL,www.ua.edu/,32177.0,1,29424.0,0.5906
...,...,...,...,...,...,...,...,...,...
1996,490513,Bais Medrash Mayan Hatorah,Lakewood,NJ,https://baismedrashmayanhatorah.com,69.0,2,16050.0,0.6471
1997,490771,Purdue University Global-St. Louis,St. Louis,MO,https://www.purdueglobal.edu/campus-locations/...,232.0,1,NaN,NaN
1998,490805,Purdue University Northwest,Hammond,IN,www.pnw.edu,8244.0,1,16949.0,0.3108
1999,490850,East-West University,Keshena,WI,www.eastwest.edu,11.0,2,40100.0,NaN


In [223]:
df_scaled

,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,ADM_RATE,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SATMTMID,SATWRMID,SAT_AVG,PCIP09,PCIP13,PCIP14,PCIP24,PCIP50,UGDS,COSTT4_A,REGION,CONTROL,HBCU,PBI,ANNHI,TRIBAL,AANAPII,HSI,NANTI,MENONLY,WOMENONLY
0,0.03125,0.166667,0.666667,0.666667,0.8986,0.500000,0.472222,0.472222,0.000000,0.584906,0.541176,0.611111,0.000000,0.0710,0.1183,0.0473,0.0237,0.062243,0.296943,0.555556,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.03125,0.000000,0.600000,0.750000,0.9211,0.694444,0.750000,0.638889,0.000000,0.698113,0.000000,0.779055,0.105693,0.0717,0.0813,0.0351,0.0339,0.164475,0.321476,0.555556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.03125,0.055556,1.000000,0.500000,0.8087,0.777778,0.833333,0.750000,0.000000,0.710692,0.000000,0.839080,0.054679,0.0283,0.2892,0.0000,0.0380,0.093027,0.309513,0.555556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.03125,0.222222,0.666667,0.666667,0.9774,0.500000,0.472222,0.472222,0.000000,0.584906,0.000000,0.620690,0.251409,0.1169,0.0000,0.0000,0.0585,0.048684,0.283568,0.555556,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.03125,0.000000,0.933333,0.833333,0.5906,0.750000,0.805556,0.694444,0.666667,0.729560,0.705882,0.799489,0.274239,0.0643,0.1160,0.0000,0.0244,0.401360,0.388513,0.555556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,0.06250,0.500000,0.733333,0.166667,0.6471,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000861,0.211923,0.222222,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1997,0.31250,0.388889,0.333333,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.002894,0.000000,0.444444,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1998,0.31250,0.166667,0.600000,0.500000,0.3108,0.611111,0.583333,0.611111,0.000000,0.616352,0.000000,0.652618,0.100620,0.0504,0.0758,0.0171,0.0000,0.102831,0.223793,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1999,0.96875,0.333333,0.666667,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.000137,0.529478,0.333333,0.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [224]:
df_final_named.iloc[290]

UNITID                     139959
INSTNM      University of Georgia
CITY                       Athens
STABBR                         GA
INSTURL               www.uga.edu
UGDS                      29474.0
CONTROL                         1
COSTT4_A                  26252.0
ADM_RATE                   0.4863
Name: 290, dtype: object

In [225]:
# Extract stats
def get_index(college):
    college_id = df_final_named[df_final_named['INSTNM'] == 'University of Georgia']  # test it out
    college_id = college_id.index.to_list()[0]
    return college_id

i = get_index(college_name)
stats = df_final_named.iloc[i]
stats['INSTNM']

'University of Georgia'